# How to create a fake network

In [8]:
import nes
import numpy as np
import os

Function to create an empty array.

In [13]:
def create_empty_array(shape):
    empty_array = np.empty(shape)
    empty_array[:] = np.nan
    return empty_array

This is where the network will be saved:

In [14]:
working_directory = os.getcwd()
working_directory

'/gpfs/scratch/bsc32/bsc032388/nes/tutorials/2.Creation'

## 1. Set Up information

### Introduce the original network directory

In [15]:
path = "/gpfs/projects/bsc32/AC_cache/obs/ghost/GHOST/1.5/hourly"

### Initialise the species, the original year and the fake network year

In [16]:
species = ['sconco3','pm2p5','pm10','sconcno2','sconcco']
og_year = 2005
fake_year = 2026

## 2. Iterate through the loop

In [17]:
for spec in species:
    # get the species directory
    spec_path = os.path.join(path,spec)
    if os.path.isdir(spec_path):
        for month in range(1,13):
            str_month = str(month).zfill(2)

            # get original nc file path
            date_path = os.path.join(spec_path, f"{spec}_{og_year}{str_month}.nc")

            print(f"\nOriginal: {date_path}")

            # load original nc file
            ness_nc_file = nes.open_netcdf(path=date_path, info=False)
            ness_nc_file.keep_vars([spec,"station_name","station_reference"])
            ness_nc_file.load()

            # get the parameters data
            lat = ness_nc_file.lat["data"]
            lon = ness_nc_file.lon["data"]
            times = ness_nc_file.time

            # get the variables data
            station_name_data = ness_nc_file.variables["station_name"]["data"]
            station_reference_data = ness_nc_file.variables["station_reference"]["data"]

            # get the variable data shape
            species_shape = ness_nc_file.variables[spec]["data"].shape
            
            # initialize nes
            nessy = nes.create_nes(comm=None, info=False, projection=None, parallel_method='X', lat=lat, lon=lon, times=times)
            nessy.variables = {'station_name': {'data': station_name_data,
                                            'dimensions': ('station',),
                                            'dtype': str},
                               'station_reference': {'data': station_reference_data,
                                            'dimensions': ('station',),
                                            'dtype': str},
                                spec: {'data': create_empty_array(species_shape).T,
                                            'dimensions': ('time', 'station',),
                                            'dtype': float}}

            # create the directory
            netcdf_path = f'fake/GHOST_2005/hourly/{spec}/'
            if not os.path.exists(os.path.dirname(netcdf_path)):
               os.makedirs(os.path.dirname(netcdf_path))

            # To run Providentia, this folder should be moved to:
            # '/esarchive/obs/' as in '/esarchive/obs/fake/GHOST_2005/hourly/species'

            # get the file name
            output_file = f'{spec}_{fake_year}{str_month}.nc'

            # save nc file
            nessy.set_strlen(75)
            nessy.to_netcdf(os.path.join(netcdf_path,output_file), info=False) 

            print(f"Saved: {os.path.join(working_directory,netcdf_path,output_file)}")

            del nessy


Original: /gpfs/projects/bsc32/AC_cache/obs/ghost/GHOST/1.5/hourly/sconco3/sconco3_200501.nc
Saved: /gpfs/scratch/bsc32/bsc032388/nes/tutorials/2.Creation/fake/GHOST_2005/hourly/sconco3/sconco3_202601.nc

Original: /gpfs/projects/bsc32/AC_cache/obs/ghost/GHOST/1.5/hourly/sconco3/sconco3_200502.nc
Saved: /gpfs/scratch/bsc32/bsc032388/nes/tutorials/2.Creation/fake/GHOST_2005/hourly/sconco3/sconco3_202602.nc

Original: /gpfs/projects/bsc32/AC_cache/obs/ghost/GHOST/1.5/hourly/sconco3/sconco3_200503.nc
Saved: /gpfs/scratch/bsc32/bsc032388/nes/tutorials/2.Creation/fake/GHOST_2005/hourly/sconco3/sconco3_202603.nc

Original: /gpfs/projects/bsc32/AC_cache/obs/ghost/GHOST/1.5/hourly/sconco3/sconco3_200504.nc
Saved: /gpfs/scratch/bsc32/bsc032388/nes/tutorials/2.Creation/fake/GHOST_2005/hourly/sconco3/sconco3_202604.nc

Original: /gpfs/projects/bsc32/AC_cache/obs/ghost/GHOST/1.5/hourly/sconco3/sconco3_200505.nc
Saved: /gpfs/scratch/bsc32/bsc032388/nes/tutorials/2.Creation/fake/GHOST_2005/hourly/s